In [1]:
import numpy as np
import exifread
from scipy import io as scipy_io
from skimage import io as skimage_io
import os

In [ ]:
# scipy_io.savemat('/Users/raymondbaranski/Desktop/test.mat', dict(counts=data))

In [15]:
def loadTIFF_folder(point_path, subfolder='TIFs'):
    data_path = point_path + '/' + subfolder + '/'
    files = os.listdir(data_path)
    tifs = list()
    for file in files:
        if file.endswith('.tif') or file.endswith('.tiff'):
            tifs.append(file)
    labels = list()
    counts = dict()
    tags = dict()
    for tif in tifs:
        label = tif.replace('.tif', '')
        labels.append(label)
        counts[label] = skimage_io.imread(data_path + '/' + tif).astype(int)
        with open(data_path + '/' + tif, 'rb') as f:
            tags[label] = exifread.process_file(f)
    point = dict()
    point['counts'] = counts
    point['labels'] = labels
    point['tags'] = tags
    return point

def estimate_lambda(point, network):
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    
    with torch.no_grad():
        network.eval()
        network.to(device)
        
        lambdas = dict()
        for label in points['labels']:
            raw_counts = torch.tensor(point['counts'][label]).unsqueeze(0).unsqueeze(0).to(device)
            lambda_estimate = network.process(raw_counts)
            lambda_estimate[lambda_estimate<0] = 0
            lambda_estimate = lambda_estimate[0,0,:,:].detach().cpu().numpy()
            lambdas[label] = lambda_estimate
        point['lambdas'] = lambdas
    return point

def savePoint_mat(point, point_path):
    scipy_io.savemat(point_path + '.mat', point)

In [16]:
point = loadTIFF_folder('/Users/raymondbaranski/Downloads/16_31773_4_8')
savePoint_mat(point, '/Users/raymondbaranski/Desktop/16_31773_4_8')

{'Image SubfileType': (0x00FE) Long=Full-resolution Image @ 667704,
 'Image ImageWidth': (0x0100) Short=2048 @ 667716,
 'Image ImageLength': (0x0101) Short=2048 @ 667728,
 'Image BitsPerSample': (0x0102) Short=8 @ 667740,
 'Image Compression': (0x0103) Short=Deflate @ 667752,
 'Image PhotometricInterpretation': (0x0106) Short=1 @ 667764,
 'Image StripOffsets': (0x0111) Long=[8, 7013, 16570, 24653, 33648, 42276, 51121, 61463, 71522, 80955, 90912, 101848, 113077, 124602, 134243, 143091, 151724, 160224, 168976, 177951, ... ] @ 668188,
 'Image Orientation': (0x0112) Short=Horizontal (normal) @ 667788,
 'Image SamplesPerPixel': (0x0115) Short=1 @ 667800,
 'Image RowsPerStrip': (0x0116) Short=32 @ 667812,
 'Image StripByteCounts': (0x0117) Long=[7005, 9557, 8083, 8995, 8628, 8845, 10342, 10059, 9433, 9957, 10936, 11229, 11525, 9641, 8848, 8633, 8500, 8752, 8975, 7824, ... ] @ 667932,
 'Image MinSampleValue': (0x0118) Short=0 @ 667836,
 'Image MaxSampleValue': (0x0119) Short=255 @ 667848,
 'I